In [1]:
import os
import re
import pandas as pd


In [2]:
data_folder = './csv_data'
total_samples = 2400

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(data_folder)
regex = re.compile('data_([a-zA-Z]*)_nodups_wobj\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(data_folder, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<13}{:<7}'.format(topic_name, row_count))


Immigration  6854   
AsianHate    1602   
Boomer       2184   
Vaccine      2671   
Mask         20149  


In [4]:
df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))

In [5]:
df_count

,topic,count,proportion_count
0,Immigration,6854,492
1,AsianHate,1602,115
2,Boomer,2184,157
3,Vaccine,2671,192
4,Mask,20149,1445


In [6]:
df_count['proportion_count'].sum()

2401

In [7]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'body_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_samples = df_samples.append(df_topic)

df_samples.reset_index(drop=True, inplace=True)
df_samples

,topic,hashtag,tweet_id,image_path,body_text,Unnamed: 0
0,Immigration,immigration,1298354080579035137,./data_Immigration/immigration/tweets4/EgSuFY5...,"@LeftIsBest10 nah, cos they are both using the...",469.0
1,Immigration,cdnpoli,1291074721195397120,./data_Immigration/cdnpoli/tweets1/EerRnVOXgAA...,@SethDKlein This is my underground version ver...,3855.0
2,Immigration,cdnpoli,1297904197569056768,./data_Immigration/cdnpoli/tweets4/EgMU7eQWoAI...,"Join Us on Sunday, August 30th, 6:30pm!\nVirtu...",4776.0
3,Immigration,refugees,1296060608635379712,./data_Immigration/refugees/tweets3/EfyIQTXXsA...,#CleanCookingIs more than preparing meals...\n...,3027.0
4,Immigration,cdnpoli,1298211948895571968,./data_Immigration/cdnpoli/tweets4/EgQs4zzXgAA...,Sounds like the Canadian version of @realDonal...,4635.0
...,...,...,...,...,...,...
2396,Mask,facemask,1293301322524037120,./data_Mask/facemask/tweets2/EfK6stXX0AUMHN4.jpg,Excited to share the latest addition to my #et...,15466.0
2397,Mask,NoMasks,1293053718292766720,./data_Mask/NoMasks/tweets2/EfHZf-4XkAEkRSF.jpg,@ShotgunHolliday 🤣 Nice! \n#CovidHoax #NoMasks...,17492.0
2398,Mask,facemask,1292890249446797317,./data_Mask/facemask/tweets2/EfFE1IsWAAIntrB.jpg,👉 https://t.co/JrYD3TuXl7\n\nThese face masks ...,15597.0
2399,Mask,WearAMask,1294605153031589888,./data_Mask/WearAMask/tweets3/EfdchVCWkAEjgqG.jpg,#WearAMask 😷 to protect yourself and others . ...,6180.0
